In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("MAT")
Pkg.add("Printf")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("JLD")
Pkg.add("Plots")

using JuMP
using Gurobi
using MAT
using Printf
using DataFrames
using CSV
using Plots
using JLD

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Ch

In [27]:
# High-level Settings
Zone = "NORTH" # price zone name

# read RTP price
fileln = matopen(string("./RTP_",Zone,"_2019_2021_1h.mat"))
RTP = read(fileln, string("RTP_",Zone,"_2019_2021_1h"))
close(fileln)
# read DAP price
fileln = matopen(string("./DAP_",Zone,"_2019_2021.mat"))
DAP = read(fileln, string("DAP_",Zone,"_2019_2021_Julia"))
close(fileln)

Spread = DAP - RTP;

# read Discharge bid price3
fileln = matopen(string("./lambda_disb3_",Zone,".mat"))
disb_price3 = read(fileln, string("lambda_disb3"))
close(fileln)

# read charge bid price3
fileln = matopen(string("./lambda_chrb3_",Zone,".mat"))
chrb_price3 = read(fileln, string("lambda_chrb3"))
close(fileln)

disb_price3 = reshape(disb_price3,(24,365))
chrb_price3 = reshape(chrb_price3,(24,365))

24×365 Matrix{Float64}:
  10.4124      3.60544      3.08047    …    0.567119  -19.2948    -8.39699
   9.79863     5.85806      2.51262        -2.49284   -19.2948    -8.39699
  11.5072      6.29906      2.05063        -6.41941   -19.2948    -8.39699
   6.71323     8.61446      1.23971        -6.41941   -19.2948    -8.39699
   3.82904    10.6078      -5.67607        -6.41941   -19.2948    -8.07365
   0.520708    9.05303     -5.67607    …   -5.6651    -19.0013    -7.4863
   0.362339   11.0384      -4.92017        -3.29072   -17.4032    -7.01981
   5.07402    10.4062      -2.84439        -0.364367  -16.7828    -6.81101
   0.889787   10.2834      -0.0467558       0.570151  -15.4221    -5.86971
  -4.65437     9.56316      3.28681         1.56866    -9.67526   -3.40822
   3.75301     7.29907      7.85954    …    0.840005  -49.923     -1.21854
  10.1547      4.38367      4.94874         1.44219   -49.923      0.8628
   9.78412     5.08885      3.12568         1.14344   -49.923      2.67082
  1

In [28]:
## battery setting

E = 1;  # storage energy capacity in MWh
# simulation setting
T = 24; # total time steps
P = .5; # power rating MW
eta = 0.9; # efficiency
e0 = .5 * E;
ef = e0;
MC = 10; # marginal discharge cost
M = 1; # duaration per step in hour
N_sim = 365; # number of days
d_DAP = DAP[:,1]; # daily DA price
d_RTP = RTP[:,1];
d_Spread = Spread[:,1];
d_disb_price3 = disb_price3[:,1];
d_chrb_price3 = chrb_price3[:,1];

# initialize optimization model
model = Model(Gurobi.Optimizer)
set_silent(model) # no outputs

@variable(model, d_da[1:T], lower_bound = 0) # discharge power in DA
# @variable(model, d_rt[1:T], lower_bound = 0) # discharge power in RT
@variable(model, c_da[1:T], lower_bound = 0) # charge power in DA
# @variable(model, c_rt[1:T], lower_bound = 0) # charge power in RT
# @variable(model, diff[1:T]) # difference between RT and DA power decision

@variable(model, e[1:T], lower_bound = 0) # total battery energy
@variable(model, R) # market revenue
# @variable(model, C) # total degradation cost
@variable(model, u_da[1:T], Bin) # 1 if charge/buy in DAM

# @constraint(model, DiffPower[t=1:T], diff[t] - d_rt[t] + c_rt[t] == -d_pDA[t] ) # actual dis/charge power 

# arbitrage revenue
@constraint(model, ArbRev, M*sum( (d_Spread .- d_disb_price3).*d_da + (d_chrb_price3 .- d_Spread).*c_da ) - R == 0 )
# @constraint(model, ArbRev, M*sum( (d_DAP .- d_disb_price2).*d_da + (d_chrb_price2 .- d_DAP).*c_da ) - R == 0 )

# total degradation cost
# @constraint(model, DegCost, M*sum(d_rt * MC) - C == 0 )
# @constraint(model, DegCost, C == M*sum(diff .* MC))

# initial SoC evolution
@constraint(model, SoCInit, e[1] - e0 == M*(c_da[1]*eta - d_da[1]/eta) )
# rest SoC evolution
@constraint(model, SoCCont[t = 2:T], e[t] - e[t-1] == M*(c_da[t]*eta - d_da[t]/eta) )

# final energy level
@constraint(model, Enelast, e[T] >= ef )
# @constraint(model, Enelast[t = 24:24:T], e[t] >= ef )

# charging / discharging non-conflict condition
@constraint(model, ChRatingTot[t=1:T], c_da[t] <= P*u_da[t])
@constraint(model, DchRatingTot[t=1:T], d_da[t] <= P*(1-u_da[t]))

# max energy level
@constraint(model, SoCMax[t=1:T], e[t] <= E )

# maximize revenue plus degradation value
@objective(model, Max, R);
# @objective(model, Max, R);
# print(model);
# ArbRev


Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


In [29]:
# initialize
R_s = zeros(1, N_sim)
P_s = zeros(1, N_sim)
C_s = zeros(1, N_sim)

C_v_da = zeros(24, N_sim) # daily DA charge
D_v_da = zeros(24, N_sim) # daily DA discharge
soc = zeros(24, N_sim) # daily SoC

# step = 0

@time begin
@printf("Optimization starts...\n")

for n = 1:(N_sim)       
    # update prices
    local pda = DAP[:,n]
    local prt = RTP[:,n]
    local pspread = Spread[:,n]
    local pdisb3 = disb_price3[:,n]
    local pchrb3 = chrb_price3[:,n]
        
    # update prices in constraints
    for t = 1:T
        set_normalized_coefficient(ArbRev, d_da[t], M*(pspread[t]-pdisb3[t]))           
        set_normalized_coefficient(ArbRev, c_da[t], -M*(pspread[t]-pchrb3[t]))     
        # set_normalized_coefficient(ArbRev, d_da[t], M*(pda[t]-pdisb2[t]))           
        # set_normalized_coefficient(ArbRev, c_da[t], -M*(pda[t]-pchrb2[t]))           
    end

    optimize!(model)
    
    global R_s[n] = value(R)
    # global C_s[n] = value(C)
    global P_s[n] = objective_value(model)
    # global P_s[n] = objective_value(model)

    global C_v_da[:,n] = value.(c_da)*M
    global D_v_da[:,n] = value.(d_da)*M    
    global soc[:,n] = value.(e)
    termination_status(model)
    # global e0 = value.(e)[end]
        
    # @printf("Day %d, Cummulative Rev %d, Cummulative Profit %d, Cummulative Cost %d, OptStatus: %s \n", n, sum(R_s), sum(P_s), sum(C_s), termination_status(model))
    # @printf("Day %d, Cummulative Rev %d, Cummulative Profit %d, OptStatus: %s \n", n, sum(R_s), sum(P_s), termination_status(model))
 
end
    
end
rtp = vec(reshape(RTP,(8760,1)))
dap = vec(reshape(DAP,(8760,1)))

DAcharge = vec(reshape(C_v_da,(8760,1)))
DAdischarge = vec(reshape(D_v_da,(8760,1)))
SoC = vec(reshape(soc,(8760,1)))
df = DataFrame(DAP = dap, RTP = rtp, DA_Charge = DAcharge, DA_Discharge = DAdischarge, SoC = SoC)
# CSV.write("DAdispatch_918.csv", df);
CSV.write(string("DA_Economic_Dispatch3_",Zone,"_2019.csv"), df);
# CSV.write(string("DA_Economic_Dispatch2_",Zone,"_2019.csv"), df);



Optimization starts...
  1.105651 seconds (616.87 k allocations: 28.730 MiB)
